In [0]:

from math import inf as infinity
from random import choice
import platform
import time
from os import system

"""
An implementation of Minimax AI Algorithm in Tic Tac Toe,
using Python.
This software is available under GPL license.
Author: Clederson Cruz
Year: 2017
License: GNU GENERAL PUBLIC LICENSE (GPL)
"""

HUMAN = -1
COMP = +1
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]


def evaluate(state):
    """
    Function to heuristic evaluation of state.
    :param state: the state of the current board
    :return: +1 if the computer wins; -1 if the human wins; 0 draw
    """
    if wins(state, COMP):
        score = +1
    elif wins(state, HUMAN):
        score = -1
    else:
        score = 0

    return score


def wins(state, player):
    """
    This function tests if a specific player wins. Possibilities:
    * Three rows    [X X X] or [O O O]
    * Three cols    [X X X] or [O O O]
    * Two diagonals [X X X] or [O O O]
    :param state: the state of the current board
    :param player: a human or a computer
    :return: True if the player wins
    """
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    if [player, player, player] in win_state:
        return True
    else:
        return False


def game_over(state):
    """
    This function test if the human or computer wins
    :param state: the state of the current board
    :return: True if the human or computer wins
    """
    return wins(state, HUMAN) or wins(state, COMP)


def empty_cells(state):
    """
    Each empty cell will be added into cells' list
    :param state: the state of the current board
    :return: a list of empty cells
    """
    cells = []

    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])

    return cells


def valid_move(x, y):
    """
    A move is valid if the chosen cell is empty
    :param x: X coordinate
    :param y: Y coordinate
    :return: True if the board[x][y] is empty
    """
    if [x, y] in empty_cells(board):
        return True
    else:
        return False


def set_move(x, y, player):
    """
    Set the move on board, if the coordinates are valid
    :param x: X coordinate
    :param y: Y coordinate
    :param player: the current player
    """
    if valid_move(x, y):
        board[x][y] = player
        return True
    else:
        return False


def minimax(state, depth, player):
    """
    AI function that choice the best move
    :param state: current state of the board
    :param depth: node index in the tree (0 <= depth <= 9),
    but never nine in this case (see iaturn() function)
    :param player: an human or a computer
    :return: a list with [the best row, best col, best score]
    """
    if player == COMP:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best


def clean():
    """
    Clears the console
    """
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')


def render(state, c_choice, h_choice):
    """
    Print the board on console
    :param state: current state of the board
    """

    chars = {
        -1: h_choice,
        +1: c_choice,
        0: ' '
    }
    str_line = '---------------'

    print('\n' + str_line)
    for row in state:
        for cell in row:
            symbol = chars[cell]
            print(f'| {symbol} |', end='')
        print('\n' + str_line)


def ai_turn(c_choice, h_choice):
    """
    It calls the minimax function if the depth < 9,
    else it choices a random coordinate.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    clean()
    print(f'Computer turn [{c_choice}]')
    render(board, c_choice, h_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, COMP)
        x, y = move[0], move[1]

    set_move(x, y, COMP)
    time.sleep(1)


def human_turn(c_choice, h_choice):
    """
    The Human plays choosing a valid move.
    :param c_choice: computer's choice X or O
    :param h_choice: human's choice X or O
    :return:
    """
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    # Dictionary of valid moves
    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'Human turn [{h_choice}]')
    render(board, c_choice, h_choice)

    while move < 1 or move > 9:
        try:
            move = int(input('Use numpad (1..9): '))
            coord = moves[move]
            can_move = set_move(coord[0], coord[1], HUMAN)

            if not can_move:
                print('Bad move')
                move = -1
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')


def main():
    """
    Main function that calls all functions
    """
    clean()
    h_choice = ''  # X or O
    c_choice = ''  # X or O
    first = ''  # if human is the first

    # Human chooses X or O to play
    while h_choice != 'O' and h_choice != 'X':
        try:
            print('')
            h_choice = input('Choose X or O\nChosen: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Setting computer's choice
    if h_choice == 'X':
        c_choice = 'O'
    else:
        c_choice = 'X'

    # Human may starts first
    clean()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    # Main loop of this game
    while len(empty_cells(board)) > 0 and not game_over(board):
        if first == 'N':
            ai_turn(c_choice, h_choice)
            first = ''

        human_turn(c_choice, h_choice)
        ai_turn(c_choice, h_choice)

    # Game over message
    if wins(board, HUMAN):
        clean()
        print(f'Human turn [{h_choice}]')
        render(board, c_choice, h_choice)
        print('YOU WIN!')
    elif wins(board, COMP):
        clean()
        print(f'Computer turn [{c_choice}]')
        render(board, c_choice, h_choice)
        print('YOU LOSE!')
    else:
        clean()
        render(board, c_choice, h_choice)
        print('DRAW!')

    exit()


if __name__ == '__main__':
    main()


Choose X or O
Chosen: X
First to start?[y/n]: y
Human turn [X]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 3
Computer turn [O]

---------------
|   ||   || X |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
|   ||   || X |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Use numpad (1..9): 6
Computer turn [O]

---------------
|   ||   || X |
---------------
|   || O || X |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
|   ||   || X |
---------------
|   || O || X |
---------------
|   ||   || O |
---------------
Use numpad (1..9): 1
Computer turn [O]

---------------
| X ||   || X |
---------------
|   || O || X |
---------------
|   ||   || O |
---------------
Human turn [X]

---------------
| X || O || X |
---------------
|   || O || X |
---------------
|   ||   || O |

In [0]:
# Python3 program to find out maximum 
# value from a given sequence of coins 

# Returns optimal value possible that 
# a player can collect from an array 
# of coins of size n. Note than n 
# must be even 
def optimalStrategyOfGame(arr, n): 
	
	# Create a table to store 
	# solutions of subproblems 
	table = [[0 for i in range(n)] 
				for i in range(n)] 

	# Fill table using above recursive 
	# formula. Note that the table is 
	# filled in diagonal fashion 
	 
	for gap in range(n): 
		for j in range(gap, n): 
			i = j - gap 
			
			# Here x is value of F(i+2, j), 
			# y is F(i+1, j-1) and z is 
			# F(i, j-2) in above recursive 
			# formula 
			x = 0
			if((i + 2) <= j): 
				x = table[i + 2][j] 
			y = 0
			if((i + 1) <= (j - 1)): 
				y = table[i + 1][j - 1] 
			z = 0
			if(i <= (j - 2)): 
				z = table[i][j - 2] 
			table[i][j] = max(arr[i] + min(x, y), 
							arr[j] + min(y, z)) 
	return table[0][n - 1] 

# Driver Code 
arr1 = [ 8, 15, 3, 7 ] 
n = len(arr1) 
print(optimalStrategyOfGame(arr1, n)) 

arr2 = [ 2, 2, 2, 2 ] 
n = len(arr2) 
print(optimalStrategyOfGame(arr2, n)) 

arr3 = [ 20, 30, 2, 2, 2, 10] 
n = len(arr3) 
print(optimalStrategyOfGame(arr3, n)) 

#The above solution can be optimized by using less number of comparisons for every choice. Please refer below.

22
4
42


In [0]:
# python program to find out maximum value from a 
# given sequence of coins 

def oSRec (arr, i, j, Sum): 

	if (j == i + 1): 
		return max(arr[i], arr[j]) 

	# For both of your choices, the opponent 
	# gives you total Sum minus maximum of 
	# his value 
	return max((Sum - oSRec(arr, i + 1, j, Sum - arr[i])), 
				(Sum - oSRec(arr, i, j - 1, Sum - arr[j]))) 

# Returns optimal value possible that a player can 
# collect from an array of coins of size n. Note 
# than n must be even 
def optimalStrategyOfGame(arr, n): 

	Sum = 0
	Sum = sum(arr) 
	return oSRec(arr, 0, n - 1, Sum) 

# Driver code 

arr1= [ 8, 15, 3, 7] 
n = len(arr1) 
print(optimalStrategyOfGame(arr1, n)) 

arr2= [ 2, 2, 2, 2 ] 
n = len(arr2) 
print(optimalStrategyOfGame(arr2, n)) 

arr3= [ 20, 30, 2, 2, 2, 10 ] 
n = len(arr3) 
print(optimalStrategyOfGame(arr3, n)) 



22
4
42


**Minmax Alpha Beta**

In [0]:


# This can either be run from a command line with python3 alphabeta.py or imported with
# from alphabeta import alphabeta

# USAGE:
# alphabeta(input, start, upper, lower)
#
# Where:
# input is a list form input tree.  See example in this file.
# start is the root node number.  So, either 0 or 1 (0 if root is MAX, 1 if root is MIN)
# upper is the upper limit for beta.  Set this to something higher than any value in your tree
# lower is the lower limit for beta.  Set this to something less than any value in your tree
#
# The function returns the root alpha and beta values, as well as the result, and the number of
# 'prunings' that took place.

# This is the tree we are working with
tree = [[[5, 1, 2], [8, -8, -9]], [[9, 4, 5], [-3, 4, 3]]]
root = 0
pruned = 0

def children(branch, depth, alpha, beta):
    global tree
    global root
    global pruned
    i = 0
    for child in branch:
        if type(child) is list:
            (nalpha, nbeta) = children(child, depth + 1, alpha, beta)
            if depth % 2 == 1:
                beta = nalpha if nalpha < beta else beta
            else:
                alpha = nbeta if nbeta > alpha else alpha
            branch[i] = alpha if depth % 2 == 0 else beta
            i += 1
        else:
            if depth % 2 == 0 and alpha < child:
                alpha = child
            if depth % 2 == 1 and beta > child:
                beta = child
            if alpha >= beta:
                pruned += 1
                break
    if depth == root:
        tree = alpha if root == 0 else beta
    return (alpha, beta)

def alphabeta(in_tree=tree, start=root, upper=-15, lower=15):
    global tree
    global pruned
    global root

    (alpha, beta) = children(tree, start, upper, lower)
    
    if __name__ == "__main__":
        print ("(alpha, beta): ", alpha, beta)
        print ("Result: ", tree)
        print ("Times pruned: ", pruned)

    return (alpha, beta, tree, pruned)

if __name__ == "__main__":
    alphabeta(None)

(alpha, beta):  5 15
Result:  5
Times pruned:  1


In [0]:
# Python3 program to demonstrate 
# working of Alpha-Beta Pruning 

# Initial values of Aplha and Beta 
MAX, MIN = 1000, -1000

# Returns optimal value for current player 
#(Initially called for root and maximizer) 
def minimax(depth, nodeIndex, maximizingPlayer, 
			values, alpha, beta): 

	# Terminating condition. i.e 
	# leaf node is reached 
	if depth == 3: 
		return values[nodeIndex] 

	if maximizingPlayer: 
	
		best = MIN

		# Recur for left and right children 
		for i in range(0, 2): 
			
			val = minimax(depth + 1, nodeIndex * 2 + i, 
						False, values, alpha, beta) 
			best = max(best, val) 
			alpha = max(alpha, best) 

			# Alpha Beta Pruning 
			if beta <= alpha: 
				break
		
		return best 
	
	else: 
		best = MAX

		# Recur for left and 
		# right children 
		for i in range(0, 2): 
		
			val = minimax(depth + 1, nodeIndex * 2 + i, 
							True, values, alpha, beta) 
			best = min(best, val) 
			beta = min(beta, best) 

			# Alpha Beta Pruning 
			if beta <= alpha: 
				break
		
		return best 
	
# Driver Code 
if __name__ == "__main__": 

	values = [3, 5, 6, 9, 1, 2, 0, -1] 
	print("The optimal value is :", minimax(0, 0, True, values, MIN, MAX)) 
 


The optimal value is : 5


In [0]:
# A simple Python3 program to find 
# maximum score that 
# maximizing player can get 
import math 

def minimax (curDepth, nodeIndex, 
			maxTurn, scores, 
			targetDepth): 

	# base case : targetDepth reached 
	if (curDepth == targetDepth): 
		return scores[nodeIndex] 
	
	if (maxTurn): 
		return max(minimax(curDepth + 1, nodeIndex * 2, 
					False, scores, targetDepth), 
				minimax(curDepth + 1, nodeIndex * 2 + 1, 
					False, scores, targetDepth)) 
	
	else: 
		return min(minimax(curDepth + 1, nodeIndex * 2, 
					True, scores, targetDepth), 
				minimax(curDepth + 1, nodeIndex * 2 + 1, 
					True, scores, targetDepth)) 
	
# Driver code 
scores = [3, 5, 2, 9, 12, 5, 23, 23] 

treeDepth = math.log(len(scores), 2) 

print("The optimal value is : ", end = "") 
print(minimax(0, 0, True, scores, treeDepth)) 




The optimal value is : 12
